In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
circuit_df = spark.read.format("delta").load(f"{processed_folder_path}/circuits").select("circuit_id",  "location")\
    .withColumnRenamed("location", "circuit_location")

In [0]:
driver_df = spark.read.format("delta").load(f"{processed_folder_path}/drivers").select("driver_id", "number", "name", "nationality")\
    .withColumnRenamed("name", "driver_name")\
    .withColumnRenamed("number", "driver_number")\
    .withColumnRenamed("nationality", "driver_nationality")\
    .withColumnRenamed("driver_id", "driv_driver_id")

In [0]:
contructors_df = spark.read.format("delta").load(f"{processed_folder_path}/constructors").select("constructor_id", "name")\
    .withColumnRenamed("name", "team")

In [0]:
race_df = spark.read.format("delta").load(f"{processed_folder_path}/races").select("race_id", "circuit_id", "name", "race_year")\
    .withColumnRenamed("name", "race_name")


In [0]:
result_df = spark.read.format("delta").load(f"{processed_folder_path}/results")\
    .filter(f"file_date = '{v_file_date}'")\
    .select("result_id", "race_id","driver_id", "constructor_id", "grid",  "points", "laps","fastest_lap_time", "time","position", "ingestion_date", "file_date")\
    .withColumnRenamed("time", "race_time")\
    .withColumnRenamed("ingestion_date", "created_date")\
    .withColumnRenamed("race_id", "re_race_id")\
    .withColumnRenamed("file_date", "result_file_date")

In [0]:
circuit_race = circuit_df.join(race_df, circuit_df.circuit_id == race_df.circuit_id, "inner")

In [0]:
circuit_race_result = circuit_race.join(result_df, circuit_race.race_id == result_df.re_race_id, "inner")

In [0]:
circuit_race_result_driver = circuit_race_result.join(driver_df, circuit_race_result.driver_id == driver_df.driv_driver_id, "inner")


In [0]:
circuit_race_result_driver_constructors = circuit_race_result_driver.join(contructors_df, circuit_race_result_driver.constructor_id == contructors_df.constructor_id, "inner")


In [0]:
df_final = circuit_race_result_driver_constructors.drop( "constructor_id", "circuit_id", "result_id", "driver_id" )


In [0]:
#overwrite_partition(df_final, "f1_presentation", "race_results", "re_race_id")

In [0]:
merge_condition = "tgt.driv_driver_id = src.driv_driver_id AND tgt.race_id = src.race_id"
merge_delta_data(df_final, "f1_presentation", "race_results", presentation_folder_path, merge_condition, "race_id")